In [1]:
## based on https://dynetworkx.readthedocs.io/en/latest/tutorial.html
import dynetworkx as dnx

import numpy as np
from numpy.random import PCG64
# use matplotlib to visualize graph?
# see https://www.geeksforgeeks.org/python-visualize-graphs-generated-in-networkx-using-matplotlib/
# %matplotlib notebook        
import matplotlib.pyplot as plt

######## IMPORT MY OWN MODULES ########
from trajectory import *          # import the simulator loop 

In [2]:
### environment constants
x_lim    = [0.2, 0.9]
y_lim    = [0., 10.]
z_lim    = [0., 2.7]

# for the fruit distribution, want to keep it the same for these tests
x_seed = PCG64(37428395352013185889194479428694397783)
y_seed = PCG64(13250124924871709375127216220749555998)
z_seed = PCG64(165440185943501291848242755689690423219)

# density of "fruit" in the orchard
density = 10


### robot constants
n_arm  = 3   # K in melon paper
n_cell = 1   # keeping it simple for now

# vehicle speed
v = 0.01   # in m/s 

# cell width/height (perpendicular to movement) and length (parallel to movement)
cell_h = z_lim[1] - z_lim[0]  # w in paper
cell_l = 0.3                  # length of cell

# arm starting locations
arm_location = np.zeros([n_arm, 3])
offset       = 0.2

arm_location[:,0] = 0. # x-coordinate start
arm_location[:,2] = 0. # z-coordinate start

for k in range(n_arm):
    # set the arms at their respective starting y-coord locations based on offset
    arm_location[k,1] = k*offset # y-coordinate start
    
# print(arm_location)

In [3]:
## Interval graph node setup
class fruitNode:
    def __init__(self, i, j, k, t): 
        # ith fruit
        self.i  = i
        # jth fruit picked by this arm
        self.j  = j
        # arm for which edges are being calculated
        self.k  = k
        # jth fruit i that the arm k has picked up, denoted t^{k}_{i(j)}, previous fruit 
        # would be t^{k}_{i(j-1)}
        # time at which kth arm reaches the jth fruit it has picked up
        self.t  = t 

In [4]:
## Functions
def calcTd(traj_calc, fruit_x, fruit_z):
    '''
       Calculate Td (handling time -> extension + retraction + conveyor drop off + pick time) 
       for node i.
    '''
    # constants
    t_grab = 0.1 
    # arm settings
    v_max = 0.8
    a_max = 3.1
    d_max = a_max
    
    # calculate extension (retraction)
    traj_calc.adjInit(0., 0.)      # starts at zero for x each time (extend to fruit)
    traj_calc.noJerkProfile(traj_calc.q0, fruit_x, traj_calc.v0, v_max, a_max, d_max)
    
    t_x = traj_calc.Ta + traj_calc.Tv + traj_calc.Td 
    
    # calculate conveyor drop off
    traj_calc.adjInit(fruit_z, 0.) # starts at fruit's location (move to conveyor from fruit)
    traj_calc.noJerkProfile(traj_calc.q0, 0., traj_calc.v0, v_max, a_max, d_max)
    
    t_z = traj_calc.Ta + traj_calc.Tv + traj_calc.Td 
    
    # add them all together
    Td = t_grab + 2*t_x + t_z
    
    return(Td)

In [5]:
## Create fruit data set
len_x = x_lim[1] - x_lim[0]            
len_y = y_lim[1] - y_lim[0]  
len_z = z_lim[1] - z_lim[0]

numFruit = int(density * (len_y*len_x*len_z))  

x = np.random.default_rng(x_seed).uniform(x_lim[0], x_lim[1], numFruit)
y = np.random.default_rng(y_seed).uniform(y_lim[0], y_lim[1], numFruit)
z = np.random.default_rng(z_seed).uniform(z_lim[0], z_lim[1], numFruit)

# need a matrix to sort x, y, and z based on the y-axis (to know what fruit show up earlier)
fruit = np.stack([x, y, z])

axis_to_sort = np.argsort(y) # sort based on y-axis
sortedFruit = fruit[:,axis_to_sort]

# create an array (or list if it'll need to be dynamic later) for node objects
node_array  = np.ndarray(numFruit+n_arm, dtype=object)  # numFruit+arm_node for the initial dummy nodes for each arm

In [6]:
## Initialize the interval graph
IG = dnx.IntervalGraph()

# arm settings, also in calcTd function
v_max = 0.8
a_max = 3.1
d_max = a_max

## initialize the ability to calculate trajectory
traj_calc = Trajectory(v_max, a_max, d_max)

In [7]:
## (a) initialize dummy 0 nodes for each arm k
for k in range(n_arm): 
    node_array[k] = fruitNode(0, 0, k, 0)
    IG.add_node(node_array[k])
    
# IG.nodes(data=True) # show the attributes for each node (maybe not necessary)
print('did the dummy node for k=1 get created?', IG.has_node(node_array[1]))
print('k value for dummy node for k=1 (should be = 1):', node_array[1].k)

for i in range(numFruit):
    node_array[i+3] = fruitNode(i, 0, 0, 0)
    IG.add_node(node_array[i+3])
    
print('number of fruit:', numFruit)    
print('number of nodes after adding all fruit:', len(IG.nodes()))

did the dummy node for k=1 get created? True
k value for dummy node for k=1 (should be = 1): 1
number of fruit: 189
number of nodes after adding all fruit: 192


In [8]:
# (b) process nodes in order of increasing y-coord
# Td = np.zeros(n_arm)
t  = np.zeros([n_arm, numFruit+1]) # t of arm k when picking fruit i
U  = np.zeros(2) # saves the interval edges

# update this for when there are more than three arms
last_i = np.array([node_array[0], node_array[1], node_array[2]]) # saves the fruit id of the fruit last picked by the arm k 
curr_j = np.array([node_array[0].j, node_array[1].j, node_array[2].j]) # saves the number of fruit being picked by 
# the kth arm

# j = 1
clique = 0 # not sure about this one

for i in range(n_arm, numFruit+n_arm): # takes into account the dummy nodes
#     print('i:', i, 'and i-3 (dummy nodes added):', i-3)
    # determine if the node i lies within the range of fewer than K preceding intervals of  
    # nodes included in the solution
    # calculate handling time for the fruit
    Td     = calcTd(traj_calc, sortedFruit[0, i-n_arm], sortedFruit[2, i-n_arm])
#     print('Td', Td)
    U[0] = sortedFruit[1, i-n_arm]   # the y-coordinate is U^{k}_{i}[0], the beginning interval edges
     
    for k in range(n_arm):
#         print('number of fruit arm', k, 'has picked:', curr_j[k])
        
        # for arm k, the previous t^{k}_{ij} is 
        prev_t = last_i[k].t
#         print('the value for t^{k}_{i(j-1)} is', prev_t, 's')
        
        # calculate the time that the kth arm reaches fruit i, the jth fruit it has picked up, at coordinate 
        # yi => they index k = 1, we index k = 0
        t[k,i-n_arm] = np.amax([prev_t+Td, (sortedFruit[1,i-n_arm]+(k)*cell_l)/v]) # or more is when it would be able to reach this next fruit
#         print('the value for the new t^{k}_{i(j)} is', t[k,i-n_arm], 's')
#         t[k,i,pot_j] = np.amax([t[k,i,pot_j-1]+Td, (sortedFruit[1,i]+(k)*cell_l)/v]) # or more is when it would be able to reach this next fruit

        # calculate U^{k}_{i}[1], the interval edge end
        U[1]   = v*(Td + t[k,i-n_arm]) - k*cell_l  # paper says it should be k*cell_w (I don't believe it)       
#         print('the beginning and end edge values are:', U)
        
        # careful with reference, the return should actually be ((u,v), begin, end)
        # https://dynetworkx.readthedocs.io/en/latest/reference/classes/generated/dynetworkx.IntervalGraph.edges.html#dynetworkx.IntervalGraph.edges
        n_interval = IG.edges(begin=U[0], end=U[1]) # check if there are edges that lie between the new edges?
        # also need to check if an interval does exist, is this arm the one in this interval (busy!)
        is_busy    = IG.edges(v=last_i[k], begin=U[0], end=U[1])
        
        print('number of intervals that fall within the current calculated edges',len(n_interval))
        print('is arm', k, 'already busy for this interval?', len(is_busy))
        
        if len(n_interval) < n_arm and len(is_busy) < 1: # it's only allowing arm 0 to pick fruit...
            # add an edge between the last node and this node with interval edge U
            print('      add edge')
            IG.add_edge(last_i[k], node_array[i], U[0], U[1])
            
            # update the node
            node_array[i].j = curr_j[k] + 1
            node_array[i].k = k
            node_array[i].t = t[k,i-n_arm]
            
            # update all the saved prev node data
            last_i[k] = node_array[i]
            curr_j[k] = node_array[i].j # maybe don't need to save this array, just use last_i
            
            # skip this fruit for the rest of the arms
            print()
            break            

print('------------------------------------------------------------------')
print('------------------------------------------------------------------')
print('Total number of fruit:', numFruit)
print('Total harvested fruit:', np.sum(curr_j))
print('Number of fruit picked by arm 0:', curr_j[0], 'arm 1:', curr_j[1], 'arm 2:', curr_j[2])

# print('U:',U)
# print()
# print('t:', t)

number of intervals that fall within the current calculated edges 0
is arm 0 already busy for this interval? 0
      add edge

number of intervals that fall within the current calculated edges 1
is arm 0 already busy for this interval? 1
number of intervals that fall within the current calculated edges 1
is arm 1 already busy for this interval? 0
      add edge

number of intervals that fall within the current calculated edges 1
is arm 0 already busy for this interval? 1
number of intervals that fall within the current calculated edges 1
is arm 1 already busy for this interval? 0
      add edge

number of intervals that fall within the current calculated edges 0
is arm 0 already busy for this interval? 0
      add edge

number of intervals that fall within the current calculated edges 0
is arm 0 already busy for this interval? 0
      add edge

number of intervals that fall within the current calculated edges 0
is arm 0 already busy for this interval? 0
      add edge

number of interv